<a href="https://colab.research.google.com/github/arpitpatelsitapur/my-py-torch-journey/blob/main/Pytorch_NN_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install torchinfo

In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchinfo import summary
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [13]:
# data preprocessing
df=pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-cancer-Detection/refs/heads/master/data.csv")
df.drop(["Unnamed: 32","id"],axis=1,inplace=True)

X=df.drop(["diagnosis"],axis=1)
y=df["diagnosis"]

le=LabelEncoder()
y=le.fit_transform(y)

sc=StandardScaler()
X=sc.fit_transform(X)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [14]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [15]:
# Define model
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))      # hidden layer with ReLU
        x = torch.sigmoid(self.fc2(x))   # output with Sigmoid
        return x

In [16]:
# Hyperparameters
input_size = 30
hidden_size = 16
num_epochs = 20
lr = 0.001

model = SimpleNN(input_size, hidden_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [18]:
summary(model)

Layer (type:depth-idx)                   Param #
SimpleNN                                 --
├─Linear: 1-1                            496
├─Linear: 1-2                            17
Total params: 513
Trainable params: 513
Non-trainable params: 0

In [19]:
# Training (whole dataset each epoch)
for epoch in range(num_epochs):
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/20], Loss: 0.6552
Epoch [2/20], Loss: 0.6481
Epoch [3/20], Loss: 0.6411
Epoch [4/20], Loss: 0.6343
Epoch [5/20], Loss: 0.6276
Epoch [6/20], Loss: 0.6210
Epoch [7/20], Loss: 0.6145
Epoch [8/20], Loss: 0.6081
Epoch [9/20], Loss: 0.6018
Epoch [10/20], Loss: 0.5957
Epoch [11/20], Loss: 0.5895
Epoch [12/20], Loss: 0.5835
Epoch [13/20], Loss: 0.5775
Epoch [14/20], Loss: 0.5716
Epoch [15/20], Loss: 0.5657
Epoch [16/20], Loss: 0.5599
Epoch [17/20], Loss: 0.5540
Epoch [18/20], Loss: 0.5483
Epoch [19/20], Loss: 0.5426
Epoch [20/20], Loss: 0.5369


In [20]:
# Testing
model.eval()
with torch.no_grad():
    outputs = model(X_test_t)
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted.eq(y_test_t).sum().item() / y_test_t.size(0)) * 100

print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 85.96%
